In [1]:
!pip install --quiet gradio

     |████████████████████████████████| 5.1 MB 22.4 MB/s 
     |████████████████████████████████| 84 kB 2.0 MB/s 
     |████████████████████████████████| 2.3 MB 29.4 MB/s 
     |████████████████████████████████| 84 kB 3.3 MB/s 
     |████████████████████████████████| 54 kB 1.5 MB/s 
     |████████████████████████████████| 57 kB 4.2 MB/s 
     |████████████████████████████████| 1.1 MB 35.6 MB/s 
     |████████████████████████████████| 140 kB 42.7 MB/s 
     |████████████████████████████████| 212 kB 48.3 MB/s 
     |████████████████████████████████| 272 kB 50.5 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 271 kB 45.0 MB/s 
     |████████████████████████████████| 144 kB 60.5 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 80 kB 7.7 MB/s 
     |████████████████████████████████| 68 kB 5.3 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |██████████████████████████

In [2]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 596 kB 28.7 MB/s 
     |████████████████████████████████| 101 kB 8.6 MB/s 
     |████████████████████████████████| 6.6 MB 51.3 MB/s 


In [3]:
import gradio as gr 
import tensorflow as tf 
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

### Loading the model and creating the generating function

In [4]:
tokenizer=GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2",pad_token_id= tokenizer.eos_token_id)

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [5]:
def generate_text(inp):
  input_ids=tokenizer.encode(inp,return_tensors='tf')
  beam_output=model.generate(input_ids,max_length=100,num_beams=5,no_repeat_ngram_size=2,early_stopping=True)
  output=tokenizer.decode(beam_output[0],skip_special_tokens=True,clean_up_tokenization_spaces=True)
  return ".".join(output.split(".")[:-1])+ "."

## Creating interface and launching

In [6]:
output_text=gr.outputs.Textbox()
gr.Interface(generate_text,"textbox",output_text,title="GPT-2",
             description="OpenAI's GPT-2 is an unsupervised language model that \
             can generate coherent text. Go ahead and input a sentence and see what it completes \
             it with! Takes around 20s to run.").launch()

/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://42422.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f88007b7550>,
 'http://127.0.0.1:7860/',
 'https://42422.gradio.app')

In [7]:
import numpy as np

import gradio as gr

def sepia(input_img):
    sepia_filter = np.array(
        [[0.393, 0.769, 0.189], [0.349, 0.686, 0.168], [0.272, 0.534, 0.131]]
    )
    sepia_img = input_img.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

demo = gr.Interface(sepia, gr.Image(shape=(200, 200)), "image")

demo.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://35695.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f87fde4f490>,
 'http://127.0.0.1:7861/',
 'https://35695.gradio.app')

In [8]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np

import gradio as gr

def sales_projections(employee_data):
    sales_data = employee_data.iloc[:, 1:4].astype("int").to_numpy()
    regression_values = np.apply_along_axis(
        lambda row: np.array(np.poly1d(np.polyfit([0, 1, 2], row, 2))), 0, sales_data
    )
    projected_months = np.repeat(
        np.expand_dims(np.arange(3, 12), 0), len(sales_data), axis=0
    )
    projected_values = np.array(
        [
            month * month * regression[0] + month * regression[1] + regression[2]
            for month, regression in zip(projected_months, regression_values)
        ]
    )
    plt.plot(projected_values.T)
    plt.legend(employee_data["Name"])
    return employee_data, plt.gcf(), regression_values

demo = gr.Interface(
    sales_projections,
    gr.Dataframe(
        headers=["Name", "Jan Sales", "Feb Sales", "Mar Sales"],
        value=[["Jon", 12, 14, 18], ["Alice", 14, 17, 2], ["Sana", 8, 9.5, 12]],
    ),
    ["dataframe", "plot", "numpy"],
    description="Enter sales figures for employees to predict sales trajectory over year.",
)
demo.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://22548.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7f87fdde0350>,
 'http://127.0.0.1:7862/',
 'https://22548.gradio.app')

## **Installing Transformers and dependancies**

In [9]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from transformers import pipeline 

## **Load Summarization pipeline**

In [11]:
summerizer= pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

## **Importing random text**

In [12]:
article= "The concept of blockchain gaming is no longer a new thing for regulators worldwide.Similar to any technology that deepened its roots in the global market, the blockchain business model attracted a plethora of investors to its side. However, every new investor who wants to take a piece of the blockchain’s cake, must know how to secure the investment and ensure it reaches its optimal potential.Just as how environmental factors have an impact on the growth of an industry, no matter how frequently we water it, a number of political factors, such as laws and regulations, severely influence the success of a certain business model. Risk may be associated with investing, but climbing into legal grey area isn’t.A 2021 report by the Blockchain Gaming Alliance demonstrated that NFT-based games have accumulated $2.32 billion in revenue between July and September in the last year, along with Metaverse-related activities exploding in the first half of 2021, especially in the United States and Asian Countries. The same report further showed that the majority of active developers in the blockchain gaming industry are concerned about the impact of government-imposed laws and regulations on the industry or the lack of it as well.Another report shed light on the gaming market’s status, predicting that it is expected to grow by $125.65 billion from 2020 to 2025 due to the integration of blockchain technology as a key market driver.Looking at these numbers suggests that the regulators are speeding up in the race to regulate the blockchain gaming market, but the reality is that they are still at the starting point.If we take a look now at the legal situations in several jurisdictions, we can see a general objective to regulate with a number of disparities. For instance, the United States is considered to be the most advanced country in the aspect of adopting blockchain and cryptocurrency regulations on the federal level and agency cooperation, along with laws that differ on a state-to-state basis.However, it is interesting that the U.S. government has been taking an active stand on regulating the cryptocurrency industry, but the shushing habit regarding other blockchain-related business models remained.On the other hand, the EU as another main regulator on the global level, has been taking on a more active role in regulating blockchain-based business models and crypto assets, but it is all still in the status of proposals. The Union adopted an extensive package of legislative proposals for the sake of regulating crypto assets and increasing investments, along with a pilot regime for market infrastructures. By moving away from cryptocurrencies and opening the blockchain-based gaming sandbox, we may encounter that most countries that brought in crypto laws didn’t quite cover the whole area."

## **Summary through summerizer with accurate length defined**

In [13]:
summary=summerizer(article,max_length=130,min_length=30,do_sample=False)

In [14]:
summary[0]['summary_text']

' The concept of blockchain gaming is no longer a new thing for regulators worldwide . A 2021 report by the Blockchain Gaming Alliance showed that NFT-based games have accumulated $2.32 billion in revenue between July and September in the last year, along with Metaverse-related activities exploding in the first half of 2021, especially in the United States and Asian countries .'

In [15]:
print("Article: \n")
print(article)
print("------------------------------------------------------------ \n")
print("Generated Summary: \n")
print(summary[0]['summary_text'])

Article: 

The concept of blockchain gaming is no longer a new thing for regulators worldwide.Similar to any technology that deepened its roots in the global market, the blockchain business model attracted a plethora of investors to its side. However, every new investor who wants to take a piece of the blockchain’s cake, must know how to secure the investment and ensure it reaches its optimal potential.Just as how environmental factors have an impact on the growth of an industry, no matter how frequently we water it, a number of political factors, such as laws and regulations, severely influence the success of a certain business model. Risk may be associated with investing, but climbing into legal grey area isn’t.A 2021 report by the Blockchain Gaming Alliance demonstrated that NFT-based games have accumulated $2.32 billion in revenue between July and September in the last year, along with Metaverse-related activities exploding in the first half of 2021, especially in the United States